## You can download an NVIDIA model from torchhub

#### ... and then compress it as a tar.gz file, and use this for deployment:

In [2]:
!wget https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt

--2020-04-24 15:54:51--  https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 54.71.37.245, 52.27.83.248
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|54.71.37.245|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://s3.us-west-2.amazonaws.com/prod-model-registry-ngc-bucket/org/nvidia/models/ssdpyt_fp32/versions/2/files/nvidia_ssdpyt_fp32_190826.pt?response-content-disposition=attachment%3B%20filename%3D%22nvidia_ssdpyt_fp32_190826.pt%22&response-content-type=application%2Foctet-stream&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEFAaCXVzLXdlc3QtMiJHMEUCIA7U%2BXJ6mzT8tb8Mo9lUxGOLx8rNi1nemb0LG%2Fo%2FFtPPAiEAsQ26nRd8H5Pnzn6LSdyWyweZaga8RmUNhQ0hx%2BOJiKIqtAMIeRACGgw3ODkzNjMxMzUwMjciDJq4RK1xUSe5HJDxvCqRA1ZFlF2tnV3TPtE%2BRIJsViqDJwCtFD7HEiyKFCoxLgMBI%2F%2FMQnXWxfJmcepE5tYhgCsSHReB560xcqfXAwdOKCIUZS5ZOdeuJoLWCQZ9Q9WDSZmswkAaqVCuqKpTkqGEzNEQaA6EzU7i8xVmH3PbW4nm8lc

In [2]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('nvidia_ssdpyt_fp32_190826.pt')

In [4]:
import sagemaker
sess = sagemaker.Session()

In [5]:
bucket = sess.default_bucket()

In [7]:
modelpath = sess.upload_data(
    path='model.tar.gz', bucket=bucket,
    key_prefix='sagemaker-pytorch/input')

In [8]:
modelpath

's3://sagemaker-us-east-1-497456752804/sagemaker-pytorch/input/model.tar.gz'

In [9]:
from sagemaker import get_execution_role
role = get_execution_role()

In [59]:
%%writefile transform_script.py

import torch
import os
from six import BytesIO
import numpy as np

def model_fn(model_dir):
    model = torch.load(os.path.join(model_dir, 'nvidia_ssdpyt_fp32_190826.pt')
    return model
                       
def input_fn(request_body, request_content_type):
    """An input_fn that loads a pickled tensor"""
    if request_content_type == 'application/python-pickle':
        return torch.load(BytesIO(request_body))
    else:
        # Handle other content-types here or raise an Exception
        # if the content type is not supported.
        pass

Overwriting transform_script.py


In [60]:
from sagemaker.pytorch.model import PyTorchModel

In [11]:
pytorch_model = PyTorchModel(model_data=modelpath, role=role,
                             entry_point='transform_script.py',
                             framework_version='1.4.0')

predictor = pytorch_model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1, wait=False)

NameError: name 'modelpath' is not defined

## Also, you can download the model from torchhub with this API

In [12]:
import torch 
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)

Using cache found in /home/ec2-user/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/ec2-user/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [13]:
!ls

Pytorch BYOM from NGC.ipynb  tmp.tar.gz		      transform_script.py
tmp			     transform_script_hub.py  u.item


## Or better, download the model from torch hub on the fly

In [16]:
%%writefile transform_script_hub.py

import torch
import os
from six import BytesIO
import numpy as np

def model_fn(model_dir):
    model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math='fp32',map_location='gpu')
    return model
                       
def input_fn(request_body, request_content_type):
    return torch.load(BytesIO(request_body))

Overwriting transform_script_hub.py


In [18]:
#PyTorchModel requires a non-empty, model_data file
from sagemaker.pytorch.model import PyTorchModel
from sagemaker import get_execution_role
role = get_execution_role()

!echo "tmp content" > tmp
!tar -zcvf ./tmp.tar.gz tmp
pytorch_model = PyTorchModel(model_data = 'file://tmp.tar.gz',
                             role=role,
                             entry_point='./transform_script_hub.py',
                             framework_version='1.4.0')

predictor = pytorch_model.deploy(instance_type='ml.p3.2xlarge',
                                 initial_instance_count=1,
                                 endpoint_name='nvidia-ssd-pytorch-gpu2')


tmp
-----------------!

In [5]:
import torch

In [6]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

Using cache found in /home/ec2-user/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [7]:
uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg',
    'http://images.cocodataset.org/val2017/000000037777.jpg',
    'http://images.cocodataset.org/val2017/000000252219.jpg'
]

In [8]:
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [9]:
tensor.cpu().numpy().shape

(3, 3, 300, 300)

In [10]:
import numpy as np
import urllib
import cv2

# METHOD #1: OpenCV, NumPy, and urllib
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [11]:
image = url_to_image('https://upload.wikimedia.org/wikipedia/commons/2/25/Postmen_Office_Room.jpg')

In [12]:
image = image.transpose((2, 0, 1)).reshape(1,image.shape[0],image.shape[1],image.shape[2])

In [15]:
predictor.predict(tensor.cpu()) #tensor.cpu().numpy()

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "'NoneType' object has no attribute 'to'". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/nvidia-ssd-pytorch-gpu2 in account 497456752804 for more information.

In [17]:
1197900/(64*3*7*7)

127.32780612244898

In [ ]:
!rm nvidia_ssdpyt_fp32_190826.pt